In [1]:
import torch
from torch import nn
from torchsummary import summary
from d2l import torch as d2l
from torch.nn import init
import numpy as np


In [2]:
class CPCNN(nn.Module):
    def __init__(self,classes=2, sampleChannel=30, sampleLength=384, N1=32, kernelLength=32,S=4):
        super(CPCNN,self).__init__()
        self.S=S
        ###########  逐点卷积  Block  #############
        self.pointwise = torch.nn.Conv2d(1, N1, (sampleChannel, 1))  # 逐点卷积-输入通道1，输出N1=16，内核是30x1
        self.depthwise = torch.nn.Conv2d(N1,N1*2, (1, kernelLength),groups=N1) 
        
        ###########  3Hz  Block  #############
        self.conv2dblock1=nn.Sequential(
            nn.Conv2d(16,16, (1, 10),groups=16),
            nn.ReLU(),
            nn.BatchNorm2d(16,track_running_stats=False),
            nn.AvgPool2d((1,344)),
        )

       
        ###########  2.5Hz  Block  #############
        self.conv2dblock2=nn.Sequential(
            nn.Conv2d(16,16, (1, 20),groups=16),
            nn.ReLU(),
            nn.BatchNorm2d(16,track_running_stats=False),
            nn.AvgPool2d((1,334)),
        )
        
        
        ###########  2Hz  Block  #############
        self.conv2dblock3=nn.Sequential(
            nn.Conv2d(16,16, (1, 34),groups=16),
            nn.ReLU(),
            nn.BatchNorm2d(16,track_running_stats=False),
            nn.AvgPool2d((1,320)),
        )
        
        ###########  1.5Hz  Block  #############
        self.conv2dblock4=nn.Sequential(
            nn.Conv2d(16,16, (1, 54),groups=16),
            nn.ReLU(),
            nn.BatchNorm2d(16,track_running_stats=False),
            nn.AvgPool2d((1,300)),
        )
        
        
        ###########  FINAL LINEAR BLOCK  #############
        
        self.fc_linear=nn.Linear(16,2)
        self.softmax_out=nn.LogSoftmax(dim=1)
        
        
    def forward(self,inputdata):
        
        intermediate = self.pointwise(inputdata)
        intermediate = self.depthwise(intermediate)
        
        """
        该位置做实验：是否需要增加BN和ACT层
        """
        
        b, c, h, w = intermediate.size() #b,64,1,353

        SPC=intermediate.view(b,self.S,c//self.S,h,w)
        
        intermediate1=self.conv2dblock1(SPC[:,0,:,:,:])
        intermediate1 = intermediate1.view(intermediate1.size()[0], -1)        
        
        intermediate2=self.conv2dblock2(SPC[:,1,:,:,:])
        intermediate2 = intermediate2.view(intermediate1.size()[0], -1)
        
        intermediate3=self.conv2dblock3(SPC[:,2,:,:,:])
        intermediate3 = intermediate3.view(intermediate1.size()[0], -1)
        
        intermediate4=self.conv2dblock4(SPC[:,3,:,:,:])
        intermediate4 = intermediate4.view(intermediate1.size()[0], -1)
        
        intermediate = intermediate1+intermediate2+intermediate3+intermediate4
        intermediate = self.fc_linear(intermediate)
        output = self.softmax_out(intermediate)
        
        return output

In [7]:
# 计算精度和召回率
def accuracy(y_pred, y_true):
    y_pred = torch.argmax(y_pred, dim=1)  # 找到预测值中每行的最大值所在的位置，作为预测类别
    correct = torch.sum(y_pred == y_true).item()  # 计算预测正确的样本数
    total = len(y_true)  # 总样本数
    acc = correct / total  # 计算精度
    return acc

def recall(y_pred, y_true, class_index):
    y_pred = torch.argmax(y_pred, dim=1)  # 找到预测值中每行的最大值所在的位置，作为预测类别
    true_positives = torch.sum((y_pred == class_index) & (y_true == class_index)).item()  # 计算真阳性样本数
    actual_positives = torch.sum(y_true == class_index).item()  # 计算实际阳性样本数
    rec = true_positives / actual_positives  # 计算召回率
    return rec

# 计算 F1 分数
def f1_score(y_pred, y_true, class_index):
    rec = recall(y_pred, y_true, class_index)
    prec = precision(y_pred, y_true, class_index)
    f1 = 2 * (prec * rec) / (prec + rec)
    return f1

# 计算 AUC-ROC
def auc_roc(y_pred, y_true):
    fpr, tpr, _ = metrics.roc_curve(y_true, y_pred[:, 1])  # 计算 ROC 曲线上的 FPR 和 TPR
    auc = metrics.auc(fpr, tpr)  # 计算 ROC 曲线下的面积 AUC
    return auc

# 计算损失函数的值
def calculate_loss(model, data_loader, criterion):
    model.eval()  # 将模型设置为评估模式，即禁用 Dropout 和 BatchNorm 层的影响
    total_loss = 0.0
    for data, target in data_loader:
        data, target = data.to(device), target.to(device)  # 将数据和标签移动到指定设备（如 GPU）上
        output = model(data)

In [6]:
net1=CPCNN().cuda()
X1=torch.randn(50,1,30,384).cuda()
summary(net1,input_size=(1,30,384))
output=net1(X1)
print(output.size())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 1, 384]             992
            Conv2d-2           [-1, 64, 1, 353]           2,112
            Conv2d-3           [-1, 16, 1, 344]             176
              ReLU-4           [-1, 16, 1, 344]               0
       BatchNorm2d-5           [-1, 16, 1, 344]              32
         AvgPool2d-6             [-1, 16, 1, 1]               0
            Conv2d-7           [-1, 16, 1, 334]             336
              ReLU-8           [-1, 16, 1, 334]               0
       BatchNorm2d-9           [-1, 16, 1, 334]              32
        AvgPool2d-10             [-1, 16, 1, 1]               0
           Conv2d-11           [-1, 16, 1, 320]             560
             ReLU-12           [-1, 16, 1, 320]               0
      BatchNorm2d-13           [-1, 16, 1, 320]              32
        AvgPool2d-14             [-1, 1

In [21]:
import torch
import scipy.io as sio
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score
import torch.optim as optim

torch.cuda.empty_cache()
torch.manual_seed(0)


def run(batch_size=25,n_epoch=11,lr=0.00128):
    torch.cuda.empty_cache()
    torch.manual_seed(0)
    #    获取数据

    filename = r'../unbalanced_dataset.mat'

    tmp = sio.loadmat(filename)
    xdata = np.array(tmp['EEGsample'])
    label = np.array(tmp['substate'])
    subIdx = np.array(tmp['subindex'])

    label.astype(int)
    subIdx.astype(int)
    samplenum = label.shape[0]
    
    channelnum = 30
    subjnum = 11
    samplelength = 3
    sf = 128
    
#     lr = 0.001
    print("当前的batch_size是：",batch_size,"n_epoch是：",n_epoch,"lr是：",lr)
    
    #   把样本中的标签赋值给ydata
    ydata = np.zeros(samplenum, dtype=np.longlong)

    for i in range(samplenum):
        ydata[i] = label[i]

    #   results存储每一个主题的精准度
    results_acc = np.zeros(subjnum)
    results_rec = np.zeros(subjnum)
    results_f1 = np.zeros(subjnum)

    # 记录画图loss
    losses = []  # 记录每次迭代后训练的loss
    eval_losses = []  # 测试的


    for i in range(1, subjnum + 1):

        #       形成训练数据
        trainindx = np.where(subIdx != i)[0]
        xtrain = xdata[trainindx]
        x_train = xtrain.reshape(xtrain.shape[0], 1, channelnum, samplelength * sf)
        y_train = ydata[trainindx]

        #       形成测试数据
        testindx = np.where(subIdx == i)[0]
        xtest = xdata[testindx]
        x_test = xtest.reshape(xtest.shape[0], 1, channelnum, samplelength * sf)
        y_test = ydata[testindx]

        train = torch.utils.data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

        #       加载CPCNN模型以处理1D EEG信号
        my_net = CPCNN().double().cuda()

        #       优化函数 和 损失函数
        optimizer = optim.Adam(my_net.parameters(), lr=lr)
        loss_class = torch.nn.NLLLoss().cuda()

        for p in my_net.parameters():
            p.requires_grad = True

        #        训练分类器
        for epoch in range(n_epoch):
            
            for j, data in enumerate(train_loader, 0):
                inputs, labels = data

                input_data = inputs.cuda()
                class_label = labels.cuda()

#                 timer.start()  # 开始计时
                my_net.zero_grad()
                my_net.train()

                class_output = my_net(input_data)
                err_s_label = loss_class(class_output, class_label)
                err = err_s_label
                err.backward()
                optimizer.step()

        #       测试结果
            my_net.train(False)
            with torch.no_grad():        
                test = torch.DoubleTensor(x_test).cuda()
                answer = my_net(test)
                probs = answer.cpu().numpy()
                preds = probs.argmax(axis=-1)
                acc = accuracy_score(y_test, preds)
                rec = recall_score(y_test, preds)
                f1 = f1_score(y_test, preds)

       
        print('第', i, '个主题的精度是：', acc)
        print('第', i, '个主题的召回率是：', rec)
        print('第', i, '个主题的F1值是：', f1)
        print('\n')
        
        results_acc[i - 1] = acc
        results_rec[i - 1] = rec
        results_f1[i - 1] = f1

    print('mean accuracy:', np.mean(results_acc),'mean Rec:', np.mean(results_rec),'mean f1:', np.mean(results_f1))

In [5]:
for i in range(1,50):
    run(batch_size=25,n_epoch=i)
    print("\n")

当前的batch_size是： 25 n_epoch是： 1 lr是： 0.00128
第 1 个主题的精度是： 0.7819148936170213
第 2 个主题的精度是： 0.6060606060606061
第 3 个主题的精度是： 0.7
第 4 个主题的精度是： 0.7635135135135135
第 5 个主题的精度是： 0.84375
第 6 个主题的精度是： 0.8373493975903614
第 7 个主题的精度是： 0.6078431372549019
第 8 个主题的精度是： 0.7575757575757576
第 9 个主题的精度是： 0.8821656050955414
第 10 个主题的精度是： 0.7685185185185185
第 11 个主题的精度是： 0.6946902654867256
mean accuracy: 0.7493983358829951


当前的batch_size是： 25 n_epoch是： 2 lr是： 0.00128
第 1 个主题的精度是： 0.8031914893617021
第 2 个主题的精度是： 0.6742424242424242
第 3 个主题的精度是： 0.7666666666666667
第 4 个主题的精度是： 0.7635135135135135
第 5 个主题的精度是： 0.8660714285714286
第 6 个主题的精度是： 0.7951807228915663
第 7 个主题的精度是： 0.5980392156862745
第 8 个主题的精度是： 0.7386363636363636
第 9 个主题的精度是： 0.8757961783439491
第 10 个主题的精度是： 0.6111111111111112
第 11 个主题的精度是： 0.6902654867256637
mean accuracy: 0.7438831455227877


当前的batch_size是： 25 n_epoch是： 3 lr是： 0.00128
第 1 个主题的精度是： 0.8457446808510638
第 2 个主题的精度是： 0.6590909090909091
第 3 个主题的精度是： 0.7733333333333333
第 4 个主题的精度是： 0.783

第 5 个主题的精度是： 0.9017857142857143
第 6 个主题的精度是： 0.8855421686746988
第 7 个主题的精度是： 0.6862745098039216
第 8 个主题的精度是： 0.7462121212121212
第 9 个主题的精度是： 0.8439490445859873
第 10 个主题的精度是： 0.7222222222222222
第 11 个主题的精度是： 0.7123893805309734
mean accuracy: 0.765121456684711


当前的batch_size是： 25 n_epoch是： 21 lr是： 0.00128
第 1 个主题的精度是： 0.8404255319148937
第 2 个主题的精度是： 0.5681818181818182
第 3 个主题的精度是： 0.8
第 4 个主题的精度是： 0.7972972972972973
第 5 个主题的精度是： 0.9241071428571429
第 6 个主题的精度是： 0.8373493975903614
第 7 个主题的精度是： 0.6372549019607843
第 8 个主题的精度是： 0.7386363636363636
第 9 个主题的精度是： 0.8535031847133758
第 10 个主题的精度是： 0.6851851851851852
第 11 个主题的精度是： 0.7743362831858407
mean accuracy: 0.7687524642293693


当前的batch_size是： 25 n_epoch是： 22 lr是： 0.00128
第 1 个主题的精度是： 0.824468085106383
第 2 个主题的精度是： 0.6287878787878788
第 3 个主题的精度是： 0.8266666666666667
第 4 个主题的精度是： 0.7905405405405406
第 5 个主题的精度是： 0.9196428571428571
第 6 个主题的精度是： 0.8554216867469879
第 7 个主题的精度是： 0.6666666666666666
第 8 个主题的精度是： 0.7083333333333334
第 9 个主题的精度是： 0.8789

第 7 个主题的精度是： 0.5784313725490197
第 8 个主题的精度是： 0.7159090909090909
第 9 个主题的精度是： 0.8184713375796179
第 10 个主题的精度是： 0.8055555555555556
第 11 个主题的精度是： 0.7522123893805309
mean accuracy: 0.7506199332977669


当前的batch_size是： 25 n_epoch是： 40 lr是： 0.00128
第 1 个主题的精度是： 0.8457446808510638
第 2 个主题的精度是： 0.553030303030303
第 3 个主题的精度是： 0.7466666666666667
第 4 个主题的精度是： 0.8378378378378378
第 5 个主题的精度是： 0.9151785714285714
第 6 个主题的精度是： 0.8795180722891566
第 7 个主题的精度是： 0.6078431372549019
第 8 个主题的精度是： 0.6893939393939394
第 9 个主题的精度是： 0.8375796178343949
第 10 个主题的精度是： 0.7870370370370371
第 11 个主题的精度是： 0.8097345132743363
mean accuracy: 0.7735967615362008


当前的batch_size是： 25 n_epoch是： 41 lr是： 0.00128
第 1 个主题的精度是： 0.8404255319148937
第 2 个主题的精度是： 0.5909090909090909
第 3 个主题的精度是： 0.8133333333333334
第 4 个主题的精度是： 0.8175675675675675
第 5 个主题的精度是： 0.9107142857142857
第 6 个主题的精度是： 0.8975903614457831
第 7 个主题的精度是： 0.5882352941176471
第 8 个主题的精度是： 0.7196969696969697
第 9 个主题的精度是： 0.7898089171974523
第 10 个主题的精度是： 0.8148148148148148
第 

### 在二分类中召回率=准确率？？

In [17]:
run(batch_size=25,n_epoch=11)

当前的batch_size是： 25 n_epoch是： 11 lr是： 0.00128
第 1 个主题的精度是： 0.8882978723404256
第 1 个主题的召回率是： 0.8617021276595744
第 1 个主题的F1值是： 0.8852459016393442


第 2 个主题的精度是： 0.803030303030303
第 2 个主题的召回率是： 0.7424242424242424
第 2 个主题的F1值是： 0.7903225806451614


第 3 个主题的精度是： 0.8666666666666667
第 3 个主题的召回率是： 0.8933333333333333
第 3 个主题的F1值是： 0.8701298701298702


第 4 个主题的精度是： 0.8175675675675675
第 4 个主题的召回率是： 0.8378378378378378
第 4 个主题的F1值是： 0.8211920529801325


第 5 个主题的精度是： 0.90625
第 5 个主题的召回率是： 0.8392857142857143
第 5 个主题的F1值是： 0.8995215311004785


第 6 个主题的精度是： 0.8975903614457831
第 6 个主题的召回率是： 0.8433734939759037
第 6 个主题的F1值是： 0.8917197452229301


第 7 个主题的精度是： 0.6568627450980392
第 7 个主题的召回率是： 0.7058823529411765
第 7 个主题的F1值是： 0.6728971962616823


第 8 个主题的精度是： 0.7462121212121212
第 8 个主题的召回率是： 0.8787878787878788
第 8 个主题的F1值是： 0.7759197324414716


第 9 个主题的精度是： 0.8630573248407644
第 9 个主题的召回率是： 0.7834394904458599
第 9 个主题的F1值是： 0.8512110726643599


第 10 个主题的精度是： 0.8240740740740741
第 10 个主题的召回率是： 0.6666666666666666


### 不平衡数据集中多指标

In [22]:
run(batch_size=47,n_epoch=14,lr=0.0015)

当前的batch_size是： 47 n_epoch是： 14 lr是： 0.0015
第 1 个主题的精度是： 0.8473684210526315
第 1 个主题的召回率是： 0.75
第 1 个主题的F1值是： 0.8323699421965318


第 2 个主题的精度是： 0.675990675990676
第 2 个主题的召回率是： 0.6363636363636364
第 2 个主题的F1值是： 0.37668161434977576


第 3 个主题的精度是： 0.6431372549019608
第 3 个主题的召回率是： 0.5388888888888889
第 3 个主题的F1值是： 0.6807017543859649


第 4 个主题的精度是： 0.765625
第 4 个主题的召回率是： 0.7027027027027027
第 4 个主题的F1值是： 0.6979865771812082


第 5 个主题的精度是： 0.8901098901098901
第 5 个主题的召回率是： 0.7946428571428571
第 5 个主题的F1值是： 0.8557692307692308


第 6 个主题的精度是： 0.8241206030150754
第 6 个主题的召回率是： 0.6982758620689655
第 6 个主题的F1值是： 0.8223350253807107


第 7 个主题的精度是： 0.6103896103896104
第 7 个主题的召回率是： 0.5145631067961165
第 7 个主题的F1值是： 0.6385542168674699


第 8 个主题的精度是： 0.7783783783783784
第 8 个主题的召回率是： 0.7954545454545454
第 8 个主题的F1值是： 0.7191780821917808


第 9 个主题的精度是： 0.8675
第 9 个主题的召回率是： 0.7070063694267515
第 9 个主题的F1值是： 0.8072727272727273


第 10 个主题的精度是： 0.926829268292683
第 10 个主题的召回率是： 0.7222222222222222
第 10 个主题的F1值是： 0.812500000

### 不平衡数据集中 balanced_accuracy_score()函数的能力

In [11]:
run(batch_size=47,n_epoch=14)

当前的batch_size是： 47 n_epoch是： 14 lr是： 0.0015
第 1 个主题的精度是： 0.8484042553191489
第 2 个主题的精度是： 0.6597796143250689
第 3 个主题的精度是： 0.7161111111111111
第 4 个主题的精度是： 0.7538937242327073
第 5 个主题的精度是： 0.8755822981366459
第 6 个主题的精度是： 0.8491379310344828
第 7 个主题的精度是： 0.6592423377117838
第 8 个主题的精度是： 0.7821810542398777
第 9 个主题的精度是： 0.8390998925323059
第 10 个主题的精度是： 0.8532986111111112
第 11 个主题的精度是： 0.7719381206512193
mean accuracy: 0.7826062682186784
